In [ ]:
# import module
from sapy import displmethod
from sapy import element
from sapy import gmsh
from sapy import structure
from sapy import plotter
from matplotlib import pyplot as plt
import numpy as np

def normalize(data, lower, upper):
    max_val = np.max(data)
    min_val = np.min(data)
    data_norm = np.array([(i - min_val)*(upper-lower)/(max_val-min_val) + lower for i in data])
    return data_norm

# generate input, output for neural network
# init params
num_bars = 10
num_exams = [500]

mesh_file = 'geo/truss_%d' % num_bars
num_outs = [2, 18]
for num_out in num_outs:
    for num_exam in num_exams:
        inp_file = "data/truss_%d_%d_%d.txt" %(num_bars, num_exam, num_out)
        inp = open(inp_file, 'w')


        # load geo from file - dictionary
        mesh = gmsh.Parse(mesh_file)

        bound = {5: [1, 1], 4: [1, 1]}

        nodal_load = {1: [0., -100.],
                     3: [0., -100.]}
        ele = element.Data()
        for j in range(num_exam):       
        
            A = []
            for i in range(num_bars): 
                area = np.random.uniform(0.1, 35.0) # range(0.1, 35.0) in2
                A.append(area)
                ele.E[i] = 10000.
                ele.A[i] = area
                ele.TYPE[i] = 'Truss'
            model = structure.Builder(mesh, ele, bound)

            U, Q = displmethod.solver(mesh, model, ele, nodal_load)
        
            # then write to file
            area = " ".join([str(a) for a in A])
            if num_out == 18:
                displace = " ".join([str(u) for u in U[:8]])  # 18 outputs: 4 point displacements and force
                force = " ".join([str(q) for q in Q])
                inp.write("%s %s %s\n" %(area, displace, force))

            else:
                inp.write("%s %f %f\n" %(area, U[1], U[3]))  # 2 outputs with critical contrainst d2 and d4
                   
        #     
        inp.close()
